# Document Retrieval using TF-IDF Weighted Rank and TF-IDF Cosine Similarity

## Imports

In [45]:
# !unzip stories

In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

# %load_ext autotime

In [2]:
title = "stories"
alpha = 0.3

## Taking all folders

In [3]:
folders = [x[0] for x in os.walk(str(os.getcwd())+'/'+title+'/')]
folders[0] = folders[0][:len(folders[0])-1]

In [4]:
folders

['/Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories',
 '/Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/FARNON',
 '/Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/SRE']

## Collecting the file names and titles

In [5]:
dataset = []

c = False

for i in folders:
    file = open(i+"/index.html", 'r')
    text = file.read().strip()
    file.close()

    file_name = re.findall('><A HREF="(.*)">', text)
    file_title = re.findall('<BR><TD> (.*)\n', text)

    if c == False:
        file_name = file_name[2:]
        c = True
        
    print(len(file_name), len(file_title))

    for j in range(len(file_name)):
        dataset.append((str(i) +"/"+ str(file_name[j]), file_title[j]))

452 452
0 0
15 15


In [6]:
len(dataset)

467

In [7]:
N = len (dataset)

In [8]:
def print_doc(id):
    print(dataset[id])
    file = open(dataset[id][0], 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    print(text)

# Preprocessing

In [9]:
def convert_lower_case(data):
    return np.char.lower(data)

In [10]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [11]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [12]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [13]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [14]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [15]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

## Extracting Data

In [16]:
processed_text = []
processed_title = []

for i in dataset[:N]:
    file = open(i[0], 'r', encoding="utf8", errors='ignore')
    text = file.read().strip()
    file.close()

    processed_text.append(word_tokenize(str(preprocess(text))))
    processed_title.append(word_tokenize(str(preprocess(i[1]))))

## Calculating DF for all words

In [17]:
DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        if w =="drugs":
            print("aaa")
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

    tokens = processed_title[i]
    for w in tokens:
        if w =="drugs":
            print("aaa")
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}
for i in DF:
    DF[i] = len(DF[i])

In [18]:
# DF

In [19]:
total_vocab_size = len(DF)

In [20]:
total_vocab_size

32352

In [21]:
total_vocab = [x for x in DF]

In [22]:
print(total_vocab[:20])

['sharewar', 'trial', 'project', 'freewar', 'need', 'support', 'continu', 'one', 'hundr', 'west', 'fifti', 'three', 'north', 'jim', 'prentic', 'copyright', 'thousand', 'nine', 'nineti', 'brandon']


In [23]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

### Calculating TF-IDF for body, we will consider this as the actual tf-idf as we will add the title weight to this.

In [24]:
doc = 0

tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf

    doc += 1

In [25]:
doc_freq("misty")

0

In [26]:
tf_idf

{(0, '4x4'): 0.003087335096410697,
 (0, 'abl'): 0.0005283149622573069,
 (0, 'abound'): 0.0018438576898819933,
 (0, 'absenc'): 0.001436078315378281,
 (0, 'access'): 0.001243477406528704,
 (0, 'accustom'): 0.0031095313121171377,
 (0, 'activ'): 0.0018535470168656444,
 (0, 'acut'): 0.001947039102894984,
 (0, 'ad'): 0.00078454689367283,
 (0, 'adapt'): 0.0016579470690715598,
 (0, 'adjust'): 0.0012887760526371001,
 (0, 'adult'): 0.0015547656560585689,
 (0, 'advanc'): 0.0009788498517945947,
 (0, 'adventuresom'): 0.003087335096410697,
 (0, 'afford'): 0.0011915206723730034,
 (0, 'ago'): 0.0005764400571559554,
 (0, 'aid'): 0.0010515047282225556,
 (0, 'air'): 0.0009679269623012499,
 (0, 'aircraft'): 0.0268335736406209,
 (0, 'airplan'): 0.0022361311367184084,
 (0, 'airport'): 0.007253037502549421,
 (0, 'airstrip'): 0.003087335096410697,
 (0, 'allow'): 0.0007205834593754925,
 (0, 'alright'): 0.0015362088802674027,
 (0, 'also'): 0.0009343048294049106,
 (0, 'although'): 0.0016702201035152697,
 (0, 'al

### Calculating TF-IDF for Title

In [27]:
doc = 0

tf_idf_title = {}

for i in range(N):
    
    tokens = processed_title[i]
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])

    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1)) #numerator is added 1 to avoid negative values
        
        tf_idf_title[doc, token] = tf*idf

    doc += 1

In [28]:
tf_idf_title

{(0, 'fifti'): 0.005434960598980563,
 (0, 'go'): 0.0002906893990853149,
 (0, 'hundr'): 0.002570392381970895,
 (0, 'jim'): 0.005269857144642146,
 (0, 'nine'): 0.0008420698058556812,
 (0, 'nineti'): 0.001312716278834434,
 (0, 'north'): 0.021919902239379185,
 (0, 'one'): 0.0003992734536048051,
 (0, 'prentic'): 0.008085184948722846,
 (0, 'thousand'): 0.0008961984314476824,
 (0, 'three'): 0.0015785688576535318,
 (0, 'west'): 0.0033256596840258424,
 (1, 'fox'): 0.11198195635330804,
 (1, 'sli'): 0.11239056533822733,
 (1, 'stori'): 0.0007682063585522353,
 (2, 'bomb'): 0.023742982378177565,
 (2, 'languag'): 0.027898190361964424,
 (2, 'parser'): 0.05635662309253824,
 (2, 'smart'): 0.014515325244714838,
 (3, 'garag'): 0.008785324492085607,
 (3, 'guy'): 0.0031995505339385936,
 (3, 'pshota'): 0.004145380786745974,
 (3, 'two'): 0.0018360457195132072,
 (4, 'day'): 0.0008890808368913132,
 (4, 'earli'): 0.0028199293637210404,
 (4, 'eighteen'): 0.0042241724258131105,
 (4, 'high'): 0.006977574311606774,


In [29]:
tf_idf[(0,"go")]

0.0002906893990853149

In [30]:
tf_idf_title[(0,"go")]

0.0002906893990853149

## Merging the TF-IDF according to weights

In [31]:
for i in tf_idf:
    tf_idf[i] *= alpha

In [32]:
for i in tf_idf_title:
    tf_idf[i] = tf_idf_title[i]

In [33]:
len(tf_idf)

344385

# TF-IDF Matching Score Ranking

In [34]:
def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    print(tokens)
    
    query_weights = {}

    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    print("")
    
    for i in query_weights[:10]:
        print(i[0], dataset[i[0]][0])

Q = matching_score(10, "They'd put flowers up. She hadn't noticed. Time wouldn't hold still. She remembered, quite clearly, that time had been a simple thing; one moment following the previous one, seconds strung out neatly like her mother's pearls laid out on the dark mahogany vanity each Sunday")


['theyd', 'put', 'flower', 'hadnt', 'notic', 'time', 'wouldnt', 'hold', 'still', 'rememb', 'quit', 'clearli', 'time', 'simpl', 'thing', 'one', 'moment', 'follow', 'previou', 'one', 'second', 'strung', 'neatli', 'like', 'mother', 'pearl', 'laid', 'dark', 'mahogani', 'vaniti', 'sunday']

400 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/sucker.txt
401 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/sunday.txt
200 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/ghost
258 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/kzap.txt
444 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/wolf7kid.txt
89 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/bulnoopt.txt
249 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/jerichms.hum
205 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/gold3ber.txt
246 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/jackbstl.txt


In [35]:
# print_doc(2)

# TF-IDF Cosine Similarity Ranking

In [36]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

### Vectorising tf-idf

In [37]:
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [38]:
def gen_vector(tokens):

    Q = np.zeros((len(total_vocab)))
    
    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [41]:
def cosine_similarity(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
        
    out = np.array(d_cosines).argsort()[-k:][::-1]

#     print(out)
    
    for i in out:
        print(i, dataset[i][0])

Q = cosine_similarity(10, "They'd put flowers up. She hadn't noticed. Time wouldn't hold still. She remembered, quite clearly, that time had been a simple thing; one moment following the previous one, seconds strung out neatly like her mother's pearls laid out on the dark mahogany vanity each Sunday")

['theyd', 'put', 'flower', 'hadnt', 'notic', 'time', 'wouldnt', 'hold', 'still', 'rememb', 'quit', 'clearli', 'time', 'simpl', 'thing', 'one', 'moment', 'follow', 'previou', 'one', 'second', 'strung', 'neatli', 'like', 'mother', 'pearl', 'laid', 'dark', 'mahogani', 'vaniti', 'sunday']
401 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/sunday.txt
200 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/ghost
258 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/kzap.txt
89 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/bulnoopt.txt
16 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/6napolen.txt
400 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/sucker.txt
418 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/timem.hac
129 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/darkness.txt
249 /Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/jerichms.hum
165 /

In [40]:
print_doc(200)

('/Users/williamscott/Desktop/IR/Assignments/Assignment 2/stories/ghost', 'Time for Flowers, by Gay Bost')
TIME FOR FLOWERS
  by Gay Bost

They'd put flowers up. She hadn't noticed. Time wouldn't hold still.
She remembered, quite clearly, that time had been a simple thing; one
moment following the previous one, seconds strung out neatly like her
mother's pearls laid out on the dark mahogany vanity each Sunday
morning. But there had been a catch . . . 

Hung around Mother's neck the catch clicked and the tidy little line 
of seconds became a never ending circle with only the catch in the 
middle. For some reason the thought of pearls gathered from the sea, 
naturally nested within the confines of oyster shells, scattered 
haphazardly about the ocean floor disturbed her.

Now they'd put up the flowers in the same careless groupings. This,
too, disturbed her. Bright yellow trumpets, their collars spread to
catch the sun, dotted the front yard in clusters of two or three, five
or six. Bunc